## Graphs in Python

We'll use the graph library NetworkX to represent graphs before passing them on to Z3. Let's see some examples on how to make graphs below.

In [ ]:
# NetworkX allows one to build graphs a number of ways, and also has functions
# to construct special classes of graphs. Here's an example with a complete graph:

K4 = nx.complete_graph(4)
nx.draw(K4) # nx.draw(.) can be used to show the graph

In [ ]:
# Here's an example with a cycle graph
C7 = nx.cycle_graph(7)
nx.draw(C7)

In [ ]:
# You can define your own graphs a number of ways. One easy way is to provide a
# list of edges like so:

edgeList = [ (0,1), (0,2), (0,3), (3,4), (4,5) ]
G = nx.Graph( edgeList )
nx.draw(G)

In [ ]:
# you can get the nodes and edges of your graph like so:
print ( G.nodes() )
print ( G.edges() )

# you can read more about NetworkX's functions here: https://networkx.org/

## Making a CLIQUE solver

Recall: for a graph $G$, a $k$-clique is a subset of vertices, $S$, of $G$ such that:

1.   Each pair of vertices in $S$ is adjacent
2.   $|S| = k$

The CLIQUE problem is defined as follows:

$$ \{ \langle G, k \rangle ~|~ G \text{ is an undirected graph with a $k$-clique }  \} $$

We will build a function that, when given graph $G$ and integer $k$, outputs a Z3 system where each solution corresponds to a clique of size at least $k$. Particularly, for a graph with $n$ vertices, we will make a system with $n$ variables (one correspond to each vertex) such that if $m$ of those variables are true, where $m \geq k$, then they correspond to an $m$-clique in $G$.

In [ ]:
def makeCLIQUESolver( G, k ):
  vertices = G.nodes() # get list of vertices

  # First, we map each vertex to a unique boolean variable

  vMap = {} # a dictionary mapping each vertex to a variable

  for v in vertices:
      var_name = 'x'+str(v)
      var_v = Bool(var_name)
      vMap[v] = var_v

  s = Solver() # initialize solver

  # We will construct a system with n variables such that if k of
  # those variables are true, they correspond to a k-clique in G

  # For each pair of vertices we will add a constraint
  for u, v in itertools.combinations(vertices, 2):

    var_u = vMap[u] # get boolean variable corresponding to vertex u
    var_v = vMap[v] # get boolean variable corresponding to vertex v

    isAdjacent = (u in G.neighbors(v)) # True if u and v are adjacent

    s.add( Or( And( var_u, var_v, isAdjacent ), True ) )
    # REPLACE THE LINE ABOVE with one that ensures u and v are both selected
    # by the solver only if they are adjacent in G

  # To ensure that at least k vertices are selected, we need another constraint

  # First, let's build the following expression:
  # sum_expr = x1 + x2 + x3 + ... + xn
  # sum_expr is the expression equal to the sum of all of our variables

  sum_expr = 0
  for v in vertices:
    var_v = vMap[v]
    sum_expr += var_v

  # using sum_expr, add a constraint below that forces at least k variables to be selected

  s.add( sum_expr <= 0 ) # REPLACE THIS LINE
  return s

Let's test our solver on a few examples.

In [ ]:
G = nx.complete_graph( 4 )
s = makeCLIQUESolver( G, 3 )
nx.draw(G, with_labels=True)
print ( s.check() )
print ( s.model() )

In [ ]:
G = nx.cycle_graph( 5 )
s = makeCLIQUESolver( G, 3 )
nx.draw( G, with_labels=True )
print ( s.check() )

In [ ]:
G = nx.cycle_graph( 5 )
s = makeCLIQUESolver( G, 2 )
nx.draw( G, with_labels=True )
print ( s.check() )